## **Predicción del Nivel de PIB utilizando Datos del Banco Mundial**

El objetivo de este proyecto es predecir el nivel de PIB de distintos países a partir de indicadores económicos, sociales y demográficos obtenidos del Banco Mundial.

El trabajo debe realizarse en tres etapas principales:

- Etapa 1: Análisis descriptivo e imputación de datos

- Etapa 2: Reducción de dimensionalidad con PCA

- Etapa 3: Modelación mediante algoritmos de clasificación

### **Etapa Previa**:

En github debe crear un repositorio llamado portfolios. Además debe crear tres ramas para cada una de las etapas. Recuerde que la rama  `main` debe contener todas las actualizaciones de su código por medio de una unión entre las ramas de las etapas junto con la rama principal.


En esta etapa inicial, se deberá configurar correctamente la estructura del proyecto utilizando GitHub como sistema de control de versiones.
- Crear un repositorio en GitHub con el nombre `portafolio`
- En el archivo `README.md` debe describir claramente la finalidad del proyecto. Puede apoyarse en las indicaciones descritas acá como referencia inicial; sin embargo, el contenido debe ser reformulado y contextualizado, de modo que el repositorio refleje un proyecto original, coherente y concebido por usted, tanto en su propósito como en su enfoque.
- A partir de la rama main, crear tres ramas, cada una asociada a una etapa del trabajo (por ejemplo: `etapa-1`, `etapa-2`, `etapa-3`). Cada rama deberá contener exclusivamente los avances correspondientes a su respectiva etapa.
- Realizar la extracción inicial de los datos desde la fuente del Banco Mundial y subir estos datos al repositorio en una carpeta denominada `main`.

**Indicaciones**
- Una vez finalizada cada etapa, los cambios desarrollados en la rama correspondiente deberán integrarse a la rama `main` mediante un proceso de unión (`merge`).
- La rama `main` debe reflejar, en todo momento, la versión más actualizada y consolidada del proyecto.


### **Etapa 1: Análisis Descriptivo e Imputación de Datos**

**Revisión general del dataset**

- Identificar el número de países, años y variables disponibles.

- Número total de observaciones

- Porcentaje de datos faltantes por variable: En caso que la variable cuente con menos de un 15% de datos NA se recomienda imputar. En caso contrario, eliminar variable.

- Identificación de outliers relevantes

- Otras observaciones relevantes.


**Indicaciones**:

- Generar una tabla de estadísticas descriptivas: media, mediana, desviación estándar, máximo, mínimo.

- Mostrar la distribución del PIB (histograma o boxplot), ya que es la variable objetivo.

- Mapa con la distribución del PIB

- Discretizar la variable dependiente `NY.GDP.MKTP.PP.KD` de acuerdo con la siguiente indicación.

    ```python
    df_wb_raw['NY.GDP.MKTP.PP.KD'] = pd.qcut(df_wb_raw['NY.GDP.MKTP.PP.KD'], q=5, labels=['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High'])

    ```

- Enviar a Github a la rama 1 el notebook ejecutado en esta etapa.

    **Nota**: Debe describir de manera clara y ordenada los pasos realizados durante el desarrollo del proyecto, incorporando una breve justificación para cada uno de ellos, de modo que se expliciten las decisiones adoptadas y su coherencia con los objetivos planteados.

    Esta indicación es válida para todas las etapas del proyecto.

In [15]:
#Cargamos la base de datos del Banco Mundial mediante Google Drive.

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive')

import my_func as fn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#Importamos la base de datos descargada desde el Banco Mundial para poder tenerla en nuestro GitHub.

import os
os.makedirs("/content/drive/MyDrive/portafolio/main", exist_ok=True)

ruta_csv = "/content/drive/MyDrive/portafolio/main/datos_banco_mundial.csv"
df_wb_raw.to_csv(ruta_csv, index=False)
print("Guardado en:", ruta_csv)

Guardado en: /content/drive/MyDrive/portafolio/main/datos_banco_mundial.csv


In [17]:
# Instalamos wbgapi para acceder a la base de datos online.
!pip install wbgapi

import pandas as pd
import wbgapi as wb
import numpy as np
from math import ceil

# Definimos la funcion para descargar los datos de my_func.py
def descargar_en_chunks(indicadores, años, chunk_size=15):
    keys = list(indicadores.keys())
    n = len(keys)

    dfs = []

    for i in range(0, n, chunk_size):
        bloque = keys[i:i+chunk_size]
        print(f"Descargando bloque {i//chunk_size + 1}: {bloque}")

        df_temp = wb.data.DataFrame(
            series=bloque,
            time=años,
            columns='series' # Los indicadores son las columnas
        )
        dfs.append(df_temp)

    if not dfs:
        return pd.DataFrame()

    df_final = pd.concat(dfs, axis=1) # Concatenar todos los DataFrames.
    df_final = df_final.reset_index() # Resetea el index para obtener 'economy' y 'time' como columnas.

    # Renombrar columnas a nombres mas legibles y eficientes.
    df_final = df_final.rename(columns=indicadores)

    # Eliminamos la fila completa si el pais o año no tienen valor en PIB.
    df_final = df_final.dropna(subset=['PIB_PPP'])

    return df_final

# Prueba de validacion, se ve el correcto funcionamiento del proceso de descarga, limpieza y renombrado de variables en pequeño.
indicadores_para_estudio = {
    "NY.GDP.MKTP.PP.KD": "PIB_PPP",
    "SP.POP.TOTL": "Poblacion_Total",
    "SI.POV.GINI": "Indice_Gini"
}


# Descarga, limpia y une todo en un DataFrame.
df_estudio = descargar_en_chunks(indicadores_para_estudio, años=range(2020, 2024))

print("Descarga de datos exitosamente!")
print(df_estudio.head(10))

Descargando bloque 1: ['NY.GDP.MKTP.PP.KD', 'SP.POP.TOTL', 'SI.POV.GINI']
Descarga de datos exitosamente!
  economy    time       PIB_PPP  Indice_Gini  Poblacion_Total
0     ABW  YR2020  3.295359e+09          NaN         108587.0
1     ABW  YR2021  3.780786e+09          NaN         107700.0
2     ABW  YR2022  4.182926e+09          NaN         107310.0
3     ABW  YR2023  4.505296e+09          NaN         107359.0
4     AFE  YR2020  2.763153e+12          NaN      694446100.0
5     AFE  YR2021  2.892377e+12          NaN      713090928.0
6     AFE  YR2022  3.005226e+12          NaN      731821393.0
7     AFE  YR2023  3.065002e+12          NaN      750491370.0
8     AFG  YR2020  1.082088e+11          NaN       39068979.0
9     AFG  YR2021  8.576755e+10          NaN       40000412.0


In [ ]:
## Etapa 1: Análisis descriptivo e imputación de datos

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Descarga de datos.
indicadores_estudio = {
    "NY.GDP.MKTP.PP.KD": "PIB_PPP",
    "SP.POP.TOTL": "Poblacion_Total",
    "SL.TLF.TOTL.IN": "Fuerza_Laboral_Total",
    "FP.CPI.TOTL.ZG": "Inflacion_Pct",
    "NE.EXP.GNFS.ZS": "Exportaciones_Pct_PIB",
    "BX.KLT.DINV.WD.GD.ZS": "Inversion_Extranjera_Pct_PIB",
    "GE.EST": "Efectividad_Gobierno",
    "SP.DYN.LE00.IN": "Esperanza_Vida_Total",
    "IT.NET.USER.ZS": "Uso_Internet_Pct",
    "SE.PRM.NENR": "Matricula_Primaria_Pct"
}

# Descargamos el rango completo (1990-2024) Elegimos estos años para una predicción optima, a pesar de que en clases solo se vio 2023. Relativamente buena cantidad de años y poca demora en cargar datos.
df_wb_raw = descargar_en_chunks(indicadores_estudio, años=range(1990, 2025))


## Filtrado de paises.

# Obtenemos la información de todas las economías
info_economias = wb.economy.info()

# Filtramos solo aquellas que no son agregados (países reales). Asi no mezclamos paises con regiones.
lista_solo_paises = [c['id'] for c in info_economias.items if c['aggregate'] == False]

# Aplicamos el filtro al DataFrame.
df_wb_raw = df_wb_raw[df_wb_raw['economy'].isin(lista_solo_paises)].reset_index(drop=True)

print(f"\n--- Revisión General del Dataset ---")
n_paises = df_wb_raw['economy'].nunique()
n_años = df_wb_raw['time'].nunique()
n_vars = len(df_wb_raw.columns) - 2 # Excluyendo economy y time
total_obs = len(df_wb_raw)

print(f"Número de países: {n_paises}")
print(f"Número de años: {n_años}")
print(f"Número de variables: {n_vars}")
print(f"Total de observaciones: {total_obs}")

## Análisis de falta de datos

missing_pct = df_wb_raw.isnull().mean() * 100
print("\n--- Porcentaje de datos faltantes por variable ---")
print(missing_pct.sort_values(ascending=False))

# Identificamos columnas > 15% , < 15%. Para eliminacion o imputacion.
cols_eliminar = missing_pct[missing_pct > 15].index.tolist()

# No eliminamos identificadores ni la variable objetivo.
for col in ['economy', 'time', 'PIB_PPP']:
    if col in cols_eliminar: cols_eliminar.remove(col)

df_etapa1 = df_wb_raw.drop(columns=cols_eliminar)
print(f"\nVariables eliminadas (>15% NA): {cols_eliminar}")

# Imputación con Mediana (para las variables que quedaron con < 15% NA)
cols_a_imputar = missing_pct[(missing_pct > 0) & (missing_pct <= 15)].index.tolist()
for col in cols_a_imputar:
    if col in df_etapa1.columns:
        df_etapa1[col] = df_etapa1[col].fillna(df_etapa1[col].median())

print(f"Variables imputadas con mediana: {cols_a_imputar}")


## Estadísticas descriptivas y análisis de rangos (outliers)

# Resumimos la distribución de las variables (tendencia central y dispersión) para contextualizar el comportamiento del dataset.

print("\n--- Tabla de Estadísticas Descriptivas ---")
stats = df_etapa1.drop(columns=['time']).describe().T
print(stats[['mean', '50%', 'std', 'min', 'max']].rename(columns={'50%': 'median'}))

# Configuramos visualualmente.

sns.set_theme(style="whitegrid")

# Figura con dos sub-gráficos para evaluar la distribución de la variable objetivo (PIB) e identificar posibles valores extremos.
# Se utiliza escala log porque el PIB suele presentar magnitudes muy diferentes entre países.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# A. Histograma con Escala Logarítmica (Evita la acumulación en valores bajos)
sns.histplot(df_etapa1['PIB_PPP'], kde=True, color='teal', log_scale=True, ax=ax1)
ax1.set_title('Distribución del PIB (Escala Logarítmica)', fontsize=13, fontweight='bold')
ax1.set_xlabel('Valor PIB (Escala Log)')
ax1.set_ylabel('Frecuencia de Países')
ax1.grid(True, which="both", linestyle='--', alpha=0.5) # Cuadrícula de precisión

# B. Boxplot por Rangos (Para identificar outliers en cada nivel y comparar la dispersión por quintiles)
# Primero discretizamos temporalmente para el gráfico si no se ha hecho

temp_cat = pd.qcut(df_etapa1['PIB_PPP'], q=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
sns.boxplot(x=temp_cat, y=df_etapa1['PIB_PPP'], palette='magma', ax=ax2)
ax2.set_yscale('log')
ax2.set_title('Dispersión del PIB por Quintiles', fontsize=13, fontweight='bold')
ax2.set_xlabel('Quintiles de Ingreso (Rango)')
ax2.set_ylabel('PIB (Escala Log)')
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## Mapa de distribución por categorías

# Visualizamos la heterogeneidad geográfica del nivel de PIB.
# Discretizamos primero para que el mapa muestre "Rangos" en lugar de un gradiente confuso
# Utilizamos el ultimo año para obtener un estado contemporaneo y evitar mezclas de periodos.
df_etapa1['PIB_CAT'] = pd.qcut(df_etapa1['PIB_PPP'],
                               q=5,
                               labels=['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High'])

ultimo_año = df_etapa1['time'].max()
df_mapa = df_etapa1[df_etapa1['time'] == ultimo_año]

# El mapa por categorías, nos da facilidad de interpretacion.
fig = px.choropleth(df_mapa,
                    locations="economy",
                    color="PIB_CAT", # Cambiamos a la variable categórica
                    hover_name="economy",
                    title=f"Mapa Global de Rangos de PIB - Año {ultimo_año}",
                    color_discrete_sequence=px.colors.qualitative.Safe,
                    category_orders={"PIB_CAT": ['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High']})

# Se agregaron parámetros para un mejor ajuste de layout.
fig.update_layout(
    margin={"r":0,"t":50,"l":0,"b":0},
    legend=dict(
        x=0.01,          # Posición horizontal
        y=0.5,           # Posición vertical
        xanchor='left'   # Anclaje de la leyenda a la izquierda
    )
)

fig.show()

## Análisis de balance de clases y cuadrícula

# Esto nos ayuda a balancear la etapa de clasificacion, ya que en caso de que existiecen desbalances se puede sesgar el entrenamiento y su evaluacion.

plt.figure(figsize=(10, 5))
ax = sns.countplot(data=df_etapa1, x='PIB_CAT', palette='viridis', edgecolor='black')

# Añadir etiquetas de datos sobre las barras para análisis exacto.
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontweight='bold')

plt.title('Distribución de Observaciones por Rango de PIB', fontsize=14, fontweight='bold')
plt.xlabel('Categoría de PIB (Variable Objetivo)')
plt.ylabel('Cantidad de Observaciones')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

print("\n--- Distribución de Clases (Frecuencia Exacta) ---")
print(df_etapa1['PIB_CAT'].value_counts())

Descargando bloque 1: ['NY.GDP.MKTP.PP.KD', 'SP.POP.TOTL', 'SL.TLF.TOTL.IN', 'FP.CPI.TOTL.ZG', 'NE.EXP.GNFS.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'GE.EST', 'SP.DYN.LE00.IN', 'IT.NET.USER.ZS', 'SE.PRM.NENR']


### **Etapa 2: Reducción de Dimensionalidad con PCA**

El dataset contiene múltiples variables macroeconómicas, demográficas y sociales, por lo que se aplicará Análisis de Componentes Principales (`PCA`) con el objetivo de reducir la dimensionalidad y capturar los principales patrones subyacentes en los datos.

- Seleccionar únicamente variables numéricas y estandarizarlas previamente.
- Aplicar `PCA` y analizar la varianza explicada por cada componente.
- Elegir el número de componentes necesarias para explicar entre 70% y 90% de la varianza total, justificando brevemente dicha elección.
- Construir un nuevo DataFrame que contenga las componentes seleccionadas, el cual será utilizado como insumo para los modelos de clasificación posteriores.
- Documentar de forma clara los pasos realizados y las decisiones metodológicas adoptadas.
